In [1]:
from load_txt import loadtxt
from cep import *
import operator
import time
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

import os
os.chdir('../tests')
import ep_test
os.chdir('../core')

In [2]:
%load_ext Cython

In [3]:
def toColorList(ep):
    coloring = list()
    for color in ep.keys():
        for node in ep[color]:
            coloring.append((node, color))
    
    coloring.sort()
    
    return [_[1] for _ in coloring]

In [4]:
n = 10000
radius = 0.01

G = nx.random_geometric_graph(n, radius)

colors = toColorList(equitablePartition(*initialize(G)))
G.number_of_edges()/n

# nx.draw_networkx(G, node_color=colors)

1.5605

In [5]:
# avg_t1, avg_t2 = 0, 0
# iters = 1

# for i in range(iters):
#     res, times = ep_test.random_geometric_check(n, radius, print_res=False)
#     avg_t1 += times[0]
#     avg_t2 += times[1]
    
#     if not res:
#         print("Failure -_-")

# print(f'{(avg_t1/iters, avg_t2/iters)}')

In [6]:
def ep_check(A, colors):
    """Function that makes sure a coloring is equitable"""
    _colors = list(colors.values())

    for c1 in _colors:
        for c2 in _colors:
            x = np.sum(A[c1][:, c2], axis=1)
            if not (x==x[0]).all():
                return False

    return True

In [7]:
%%cython

def ep_check_cython(int[:, :] A, int[:, :] colors, int[:] color_inds):

    cdef int n = color_inds.shape[0]
    cdef int[:] class1, class2
    cdef int temp_sum0, temp_sum, a, b
    
    for cind1 in range(n):
        class1 = colors[cind1, :]
        
        for cind2 in range(n):
            class2 = colors[cind2, :]
        
            for c1 in range(color_inds[cind1]):
                temp_sum = 0
                a = class1[c1]
                for c2 in range(color_inds[cind2]):
                    b = class2[c2]
                    temp_sum += A[a, b]

                if c1 == 0:
                    temp_sum0 = temp_sum
                else:
                    if temp_sum != temp_sum0:
                        return False
    return True

In [37]:
n = 10
radius = 0.75

G = nx.random_geometric_graph(n, radius)
color_dict = equitablePartition(*initialize(G))

color_inds = np.array([len(x) for x in color_dict.values()]).astype(np.int32)
A = nx.to_numpy_array(G).astype(np.int32).T
colors = np.zeros((len(color_inds), np.max(color_inds)), dtype=np.int32)

for key in color_dict.keys():
    colors[key, :color_inds[key]] = color_dict[key]
    
G.number_of_edges()/n

3.5

In [41]:
%timeit ep_check_cython(A, colors, color_inds)

16.7 µs ± 503 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [39]:
%timeit ep_check(A, color_dict)

489 µs ± 3.76 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [40]:
nx.erdos_renyi_graph?

Signature: nx.erdos_renyi_graph(n, p, seed=None, directed=False)
Docstring:
Returns a $G_{n,p}$ random graph, also known as an Erdős-Rényi graph
or a binomial graph.

The $G_{n,p}$ model chooses each of the possible edges with probability $p$.

The functions :func:`binomial_graph` and :func:`erdos_renyi_graph` are
aliases of this function.

Parameters
----------
n : int
    The number of nodes.
p : float
    Probability for edge creation.
seed : integer, random_state, or None (default)
    Indicator of random number generation state.
    See :ref:`Randomness<randomness>`.
directed : bool, optional (default=False)
    If True, this function returns a directed graph.

See Also
--------
fast_gnp_random_graph

Notes
-----
This algorithm [2]_ runs in $O(n^2)$ time.  For sparse graphs (that is, for
small values of $p$), :func:`fast_gnp_random_graph` is a faster algorithm.

References
----------
.. [1] P. Erdős and A. Rényi, On Random Graphs, Publ. Math. 6, 290 (1959).
.. [2] E. N. Gilbert, R